In [108]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
import scipy.optimize as opt

pts = np.loadtxt('NewMacroSeries.txt', delimiter =',')
c, k, w, r, y = pts[:,0], pts[:,1], pts[:,2], pts[:,3], pts[:,4]

S = 1000
T = 100
k_mean = k.mean()
sim = np.random.uniform(0,1,(T,S))
print (sim)

[[ 0.27987511  0.29443092  0.84496998 ...,  0.81093766  0.30506747
   0.13076803]
 [ 0.56015995  0.60411242  0.18330272 ...,  0.97751092  0.57659554
   0.32733484]
 [ 0.9358018   0.12847279  0.27209849 ...,  0.40566121  0.62602991
   0.33949541]
 ..., 
 [ 0.86937317  0.41075325  0.21509459 ...,  0.83258206  0.52757456
   0.82684081]
 [ 0.35264298  0.60719805  0.96744723 ...,  0.93353402  0.40030778
   0.66057052]
 [ 0.50256954  0.56226379  0.96342618 ...,  0.49719909  0.00671537
   0.24989346]]


In [109]:
def simulation(alpha, beta, rho, mu, sig, sim):
    epsilon = sts.norm.ppf(sim, 0, sig)
    z = np.zeros((T,S))
    z_t_1 = mu
    for i in range(T):
        z_t = rho*z_t_1 + (1-rho)*mu + epsilon[i,:] 
        z[i,:] = z_t
        z_t_1 = z_t
            
    k = np.zeros((T,S))
    k_1 = np.zeros((T,S))
    k_t_1 = k_mean
    for i in range(T):
        k_t = alpha*beta*np.exp(z[i,:])*k_t_1**alpha
        k_1[i,:] = k_t_1
        k[i,:] = k_t
        k_t_1 = k_t
    
    w = (1-alpha)*np.exp(z)*k_1**alpha
    r = alpha*np.exp(z)*k_1**(alpha-1)
    c = w + r*k_1 - k 
    y = np.exp(z)*k_1**alpha
    
    return c, k_1, k, w, r, y

In [110]:
#Data moments
moment1 = c.mean()
moment2 = k.mean()
moment3 = (c/y).mean()
moment4 = y.std()
moment5 = np.corrcoef(c[0:99], c[1:100])[0,1]
moment6 = np.corrcoef(c, k)[0,1]
data_moments = np.array([moment1, moment2, moment3, moment4, moment5, moment6])

#Model moments
def model_moments(alpha, beta, rho, mu, sig, sim):
        c,k_1,k,w,r,y = simulation(alpha, beta, rho, mu, sig, sim)
        moment1 = c.mean()
        moment2 = k.mean()
        moment3 = (c/y).mean()
        moment4 = np.std(y,axis=0).mean()
        corr_c = np.zeros((1,S))
        corr_ck = np.zeros((1,S))
        
        for i in range(S):
            corr_c[0,i] = np.corrcoef(c[0:99,i],c[1:100,i])[0,1]
            corr_ck[0,i] = np.corrcoef(c[:,i],k_1[:,i])[0,1]
            
        moment5 = corr_c.mean()
        moment6 = corr_ck.mean()
        model_moments = np.array([moment1,moment2, moment3, moment4, moment5, moment6])
        return model_moments
    
def err_vec(alpha, beta, rho, mu, sig, sim):
    model_mmt = model_moments(alpha, beta, rho, mu, sig, sim)
    err_vec = (model_mmt - data_moments) / data_moments
    return err_vec

def criterion(params, *args):
    alpha, rho, mu, sig = params
    beta, sim = args
    err = err_vec(alpha, beta, rho, mu, sig, sim)
    W = np.eye(6)
    criterion = err @ W @ err.T
    return criterion

In [112]:
alpha = 0.2
beta = 0.99
rho = 0.8
mu = 9
sig = 0.9

params_init = np.array([alpha, rho, mu, sig])
bnds = ((1e-10, 1-1e-10),(-1+1e-10, 1-1e-10),(5,14),(1e-10,None))
result = opt.minimize(criterion, params_init, args = (beta,sim), method = 'L-BFGS-B', bounds = bnds)
alpha_smm, rho_smm, mu_smm, sig_smm = result.x

print(result)
print('alpha_smm=',alpha_smm,'rho_smm=', rho_smm,'mu_smm=', mu_smm,'sig_smm=', sig_smm)

      fun: 4.3775638550740248e-06
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -5.58401481e-05,  -1.12907462e-05,  -2.28105329e-06,
        -2.43860247e-05])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 515
      nit: 71
   status: 0
  success: True
        x: array([ 0.42103712,  0.92116011,  9.91870435,  0.09362076])
alpha_smm= 0.421037118161 rho_smm= 0.92116011107 mu_smm= 9.91870434861 sig_smm= 0.0936207566999
